In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
diabetes_dataset = pd.read_csv('diabetes.csv') 

In [3]:
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Verdiğiniz veri setindeki sütunlar ve bu sütunların anlamı hakkında genel bir yorum yapalım:

### 1. **Pregnancies (Hamilelik Sayısı)**
   - Bu sütun, bireyin geçmişteki hamilelik sayısını gösterir. Diyabetin belirli bir türü, özellikle gestasyonel diyabet, hamilelik sırasında gelişebilir. Ayrıca, hamilelik sayısı arttıkça, bazı sağlık problemleri (örneğin, yüksek kan şekeri) daha sık görülebilir.

### 2. **Glucose (Kan Şekeri Seviyesi)**
   - Kan şekeri seviyesi, diyabetin en önemli göstergelerinden biridir. Normalde kan şekeri seviyesinin yüksek olması, diyabetin bir belirtisi olabilir. Bu sütun, kişilerin diyabet riskini değerlendirmede kritik rol oynar.

### 3. **BloodPressure (Kan Basıncı)**
   - Yüksek kan basıncı (hipertansiyon) genellikle diyabetle ilişkilidir. Kan basıncı değerleri, kalp ve damar hastalıklarıyla birlikte diyabet riskini gösterebilir.

### 4. **SkinThickness (Cilt Kalınlığı)**
   - Cilt altı yağ dokusu ile ilişkili bir ölçümdür ve insulin direnci ile bağlantılı olabilir. Cilt kalınlığı yüksek olan kişilerde genellikle insulin direnci artar, bu da diyabet riskini artırır.

### 5. **Insulin (İnsülin Seviyesi)**
   - İnsülin, pankreas tarafından üretilen bir hormondur ve vücudun kan şekeri seviyelerini düzenler. İnsülin direnci olan kişilerde, insülin seviyeleri yükselir ve bu durum diyabetin bir belirtisi olabilir.

### 6. **BMI (Vücut Kitle İndeksi)**
   - BMI, bireyin kilo ve boy oranına dayalı olarak vücut yağ oranını ölçen bir değerdir. Yüksek BMI, obezite ile ilişkilidir ve obezite, diyabetin önemli bir risk faktörüdür.

### 7. **DiabetesPedigreeFunction (Diyabet Soybilim Fonksiyonu)**
   - Bu özellik, bir kişinin aile geçmişine dayalı olarak diyabet riski hakkında bilgi verir. Ailede diyabet öyküsü olan bireylerin diyabet geliştirme olasılığı daha yüksektir.

### 8. **Age (Yaş)**
   - Yaş, diyabetin gelişimiyle doğrudan ilişkilidir. Genellikle, yaş ilerledikçe diyabet riski artar. Özellikle 45 yaş ve üzerindeki bireylerde diyabet riski önemli ölçüde artar.

### 9. **Outcome (Sonuç)**
   - Bu sütun, bireyin diyabetli olup olmadığını belirtir. 1, diyabeti olan bireyleri, 0 ise diyabeti olmayanları gösterir. Modelin amacı, bu sonuca göre tahmin yapmaktır.

---



In [4]:
diabetes_dataset.shape

(768, 9)

In [5]:
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Verdiğiniz istatistiksel özet, veri setindeki her bir özelliğin dağılımını ve temel ölçümleri hakkında önemli bilgiler sunuyor. Aşağıda her bir sütunun açıklamaları ile birlikte yapılan yorumları bulabilirsiniz:

### 1. **Pregnancies (Hamilelik Sayısı)**
   - **Ortalama**: 3.85 (yaklaşık 4 hamilelik) — Genelde veriler, hamilelik sayısının çok yüksek olmadığını gösteriyor. 
   - **Standart Sapma**: 3.37 — Hamilelik sayısındaki varyasyonun oldukça geniş olduğunu gösteriyor. Bazı bireylerin çok sayıda hamileliği olabilecekken, diğerlerinin hiç hamileliği olmamış.
   - **Min/Max**: 0 ile 17 arasında değişiyor. Bu, çok geniş bir dağılım olduğunu ve bazı kişilerin hiç hamile kalmadığını, bazılarının ise çok sayıda hamileliği olduğunu gösteriyor.

### 2. **Glucose (Kan Şekeri Seviyesi)**
   - **Ortalama**: 120.89 — Ortalama kan şekeri seviyesi, genellikle diyabetli bireylerde daha yüksek olur. Bu değer, diyabetli ve diyabetsiz bireylerin birleşiminden kaynaklanmaktadır.
   - **Standart Sapma**: 31.97 — Kan şekeri seviyesinde oldukça büyük bir varyasyon olduğu gösteriyor. Bu, yüksek kan şekeri seviyelerine sahip kişilerin olduğunu ve bunun modelin doğruluğunu etkileyebileceğini gösterir.
   - **Min/Max**: Kan şekeri seviyesi 0 ile 199 arasında değişiyor. Ancak, 0 değeri genellikle eksik veri ya da anormal bir ölçüm olabilir.

### 3. **BloodPressure (Kan Basıncı)**
   - **Ortalama**: 69.11 — Bu değer, çoğu kişinin normal kan basıncı aralığında olduğunu gösteriyor. Ancak, bazı kişilerde yüksek kan basıncı olduğu da açıkça gözlemlenebilir.
   - **Standart Sapma**: 19.36 — Kan basıncı değerlerinde önemli bir varyasyon olduğunu gösteriyor.
   - **Min/Max**: 0 ile 122 arasında değişiyor. Buradaki 0 değeri, verinin eksik ya da hatalı olabileceğini gösteriyor.

### 4. **SkinThickness (Cilt Kalınlığı)**
   - **Ortalama**: 20.54 — Cilt kalınlığı, çoğu bireyde ortalama değerin civarındadır. 
   - **Standart Sapma**: 15.95 — Bu özelliğin varyasyonu oldukça geniş. Birçok bireyde 0 değerinin bulunduğu gözlemlenebilir (örneğin, cilt kalınlığı ölçülmemiş olabilir).
   - **Min/Max**: 0 ile 99 arasında değişiyor. Buradaki 0 değeri eksik veri olabilir.

### 5. **Insulin (İnsülin Seviyesi)**
   - **Ortalama**: 79.80 — Ortalama insülin seviyesi, çoğu bireyin normallere yakın olduğunu gösteriyor.
   - **Standart Sapma**: 115.24 — İnsülin seviyelerinde büyük bir varyasyon olduğu anlamına geliyor. Bu, yüksek insülin değerlerine sahip bireylerin bulunduğunu ve bunun diyabetin belirtisi olabileceğini gösteriyor.
   - **Min/Max**: 0 ile 846 arasında değişiyor. Buradaki 0 değeri eksik veya ölçülmemiş veri anlamına gelebilir. Bu nedenle, verinin temizlenmesi gerekebilir.

### 6. **BMI (Vücut Kitle İndeksi)**
   - **Ortalama**: 31.99 — Ortalama BMI, genellikle fazla kilolu veya obez bireyleri gösteriyor. Obezite, diyabet için önemli bir risk faktörüdür.
   - **Standart Sapma**: 7.88 — BMI değerleri arasında büyük bir varyasyon olduğu gözlemleniyor.
   - **Min/Max**: 0 ile 67.1 arasında değişiyor. Buradaki 0 değeri, eksik veya hatalı veri olabileceğini düşündürmektedir.

### 7. **DiabetesPedigreeFunction (Diyabet Soybilim Fonksiyonu)**
   - **Ortalama**: 0.47 — Aile geçmişi ile ilgili risk, genellikle düşük değerlerde. Ancak, bazı kişilerin bu değeri çok yüksek olabiliyor, bu da genetik yatkınlık anlamına gelir.
   - **Standart Sapma**: 0.33 — Bu fonksiyon, genetik yatkınlıkla ilgili verilerde orta düzeyde varyasyon olduğunu gösteriyor.
   - **Min/Max**: 0.078 ile 2.42 arasında değişiyor. Bu da, bazı bireylerin çok düşük genetik yatkınlık gösterdiğini, bazıların ise çok yüksek bir risk taşıdığını gösteriyor.

### 8. **Age (Yaş)**
   - **Ortalama**: 33.24 — Yaş, genellikle diyabet riskiyle doğru orantılıdır. Bu yaş ortalaması, diyabet riski taşıyan bireylerin yaşlarını göstermektedir.
   - **Standart Sapma**: 11.76 — Yaş dağılımı geniştir, yani bu veri setindeki bireylerin büyük bir kısmı genç, ancak yaşlı bireyler de bulunmaktadır.
   - **Min/Max**: 21 ile 81 arasında değişiyor. Bu, veri setindeki bireylerin geniş bir yaş aralığını kapsadığını gösteriyor.

### 9. **Outcome (Sonuç)**
   - **Ortalama**: 0.35 — Sonuç sütununda, diyabetli bireylerin oranı yaklaşık %35, diyabetsiz bireylerin oranı ise %65 civarındadır.
   - **Standart Sapma**: 0.48 — Bu değer, sınıflandırmanın oldukça dengeli olduğunu gösteriyor (sınıflar arasında çok büyük bir dengesizlik yok).

---

In [6]:
diabetes_dataset['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [7]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [8]:
X = diabetes_dataset.drop(columns = 'Outcome', axis=1)
Y = diabetes_dataset['Outcome']

In [9]:
print(X)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [10]:
print(Y)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


In [28]:
scaler = StandardScaler()

scaler.fit(X)

StandardScaler()

In [29]:
standardized_data = scaler.transform(X)

print(standardized_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [15]:
X = standardized_data
Y = diabetes_dataset['Outcome']

In [16]:
print(X)
print(Y)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [18]:
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


In [19]:
classifier = svm.SVC(kernel='linear')

In [20]:
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

Bu örnekte, **SVC (Support Vector Classification)** kullanarak bir **ikili sınıflandırma** yapılmıştır. Hedef, **diyabet hastalığı** olup olmadığını (Outcome sütununa göre) tahmin etmek. İşlem adımlarını ve yapılan sınıflandırmayı açıklayalım:

### 1. Veri Hazırlığı:
- `X`: Bağımsız değişkenleri (özellikleri) içeriyor; burada her bir satır, bir kişinin çeşitli sağlık ölçümleri ve bilgilerini (örneğin, `Glucose`, `BMI`, `Age` vs.) içeriyor.
- `Y`: Bağımlı değişkeni içeriyor; burada her bir satır, kişinin diyabet olup olmadığını (1: Hastalık var, 0: Hastalık yok) gösteriyor.

### 2. Veri Standartlaştırma:
Verilerin **ölçeklendirilmesi** önemli çünkü bazı algoritmalar, özellikle SVC gibi algoritmalar, büyük farklılıklar gösteren özelliklerden olumsuz etkilenebilir. Bu nedenle, **StandardScaler** kullanılarak veriler sıfır ortalama ve birim varyansa dönüştürülmüştür. Bu, her özelliğin aynı ölçekte olması gerektiği anlamına gelir.

- `X_train` ve `X_test` üzerinde standartlaştırma yapılmıştır.

### 3. Eğitim ve Test Verilerine Ayırma:
Veri, eğitim (train) ve test (test) setlerine ayrıldı. Burada, **%90** eğitim verisi ve **%10** test verisi kullanıldı. Bu, modelin doğruluğunu test etmek için yaygın bir tekniktir.

- `X_train` ve `Y_train`: Eğitim seti.
- `X_test` ve `Y_test`: Test seti.

Eğitim ve test verilerinin **dengeli (stratified)** olmasına dikkat edilmiştir. Yani, test setinde her iki sınıfın (diyabetli ve diyabetsiz) oranı, eğitim setindeki oranlarla aynı olacak şekilde ayrılmıştır.

### 4. Modelin Eğitilmesi:
- **SVC (Destek Vektör Makinesi)** sınıflandırıcı, doğrusal bir kernel kullanarak eğitim verileriyle eğitilmiştir.
  - `classifier = svm.SVC(kernel='linear')`: Bu satırda, doğrusal bir kernel kullanan bir **SVC sınıflandırıcı** oluşturuluyor.
  - `classifier.fit(X_train, Y_train)`: Model, eğitim verileri (`X_train`, `Y_train`) ile eğitilir.

### 5. Modelin Çalışma Prensibi:
- **SVC**, sınıflar arasında **en geniş marjı** bulmaya çalışır. Yani, verileri iki sınıfa ayıran en uygun sınırı (hiper düzlemi) belirler. Burada, sınıflandırma yapılacak olan iki sınıf (diyabetli ve diyabetsiz) arasındaki en geniş mesafeyi bulan doğrusal bir sınır belirlenir.

### Sonuç:
- Bu model, kişilerin diyabet olup olmadığını tahmin etmeye çalışır. Eğitim veri seti kullanılarak model eğitildi ve test veri seti ile performansı değerlendirilecektir.
- Modelin başarısı, test seti üzerinden doğruluk, precision, recall gibi metriklerle ölçülüp değerlendirilebilir.

### Özet:
SVC ile yapılan sınıflandırma, kişilerin sağlık verilerine (glukoz, yaş, BMI, vs.) dayanarak diyabet olup olmadıklarını tahmin etmeye yöneliktir. Doğrusal bir kernel ile iki sınıf arasında en uygun ayırıcı çizgiyi (hiper düzlemi) bulmak için eğitim yapılmıştır.

In [25]:
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.7866449511400652


In [24]:
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.7727272727272727


Bu çıktı, modelin eğitim verisi ve test verisi üzerinde ne kadar başarılı olduğunu gösteriyor. Modelin doğruluğunu (accuracy) hem eğitim verisi (`X_train`) hem de test verisi (`X_test`) üzerinde ölçtük ve her iki set için ayrı ayrı doğruluk oranlarını hesapladık. Şimdi her iki sonucu yorumlayalım:

### Eğitim Verisi Doğruluğu:
- **Eğitim verisi doğruluğu**: 0.7866 (yaklaşık %79)
  - Bu, modelin eğitim verisi üzerinde **%79** oranında doğru tahmin yaptığı anlamına gelir. Eğitim setindeki örnekleri doğru sınıflandırma oranı oldukça iyi. Bu, modelin veriyi yeterince iyi öğrendiğini ve doğru sınıflandırmalar yaptığını gösteriyor.
  
### Test Verisi Doğruluğu:
- **Test verisi doğruluğu**: 0.7727 (yaklaşık %77)
  - Test setindeki doğruluk ise **%77**. Bu, eğitim verileri üzerinde elde edilen başarıya çok yakın ancak biraz daha düşük. Bu durum, modelin eğitim setine aşırı uyum sağlamadığını (overfitting) ve genel bir başarı sağladığını gösteriyor. Eğitim verisi doğruluğu biraz daha yüksek olsa da, bu fark genellikle modelin iyi genellenebilirliğe sahip olduğunu gösterir.

### Yorum:
- **Modelin genellenebilirliği**: Modelin eğitim verisi ve test verisi üzerindeki doğrulukları çok benzer. Bu, modelin **aşırı uyum (overfitting)** yapmadığını ve **iyi genellenebilirlik** sağladığını gösterir.
- **Başarı oranı**: %79'luk bir doğruluk, genellikle sınıflandırma görevleri için iyi bir başarı oranıdır. Ancak daha yüksek doğruluk oranlarına ulaşmak için model üzerinde ek optimizasyonlar veya başka algoritmalar kullanılabilir.

Sonuç olarak, modelin hem eğitim verisi hem de test verisi üzerinde **iyi bir performans gösterdiğini** söyleyebiliriz. Yine de, doğruluğun artırılması için **özellik mühendisliği, parametre ayarlamaları veya farklı algoritmalar** gibi geliştirme yolları denenebilir.

In [35]:
# X_train veri setini alalım (veya X_test, hangisini kullanmak isterseniz)
X_data = X_train  # ya da X_test

# Veriyi standardize etme
X_data_standardized = scaler.transform(X_data)

# Modelin tüm veri seti üzerinde tahmin yapması
predictions = classifier.predict(X_data_standardized)

# Her bir tahmin sonucunu yazdırma
for i, prediction in enumerate(predictions):
    if prediction == 0:
        print(f"Row {i+1}: The person is not diabetic")
    else:
        print(f"Row {i+1}: The person is diabetic")


Row 1: The person is not diabetic
Row 2: The person is not diabetic
Row 3: The person is diabetic
Row 4: The person is not diabetic
Row 5: The person is diabetic
Row 6: The person is diabetic
Row 7: The person is not diabetic
Row 8: The person is diabetic
Row 9: The person is not diabetic
Row 10: The person is not diabetic
Row 11: The person is diabetic
Row 12: The person is diabetic
Row 13: The person is not diabetic
Row 14: The person is not diabetic
Row 15: The person is not diabetic
Row 16: The person is not diabetic
Row 17: The person is not diabetic
Row 18: The person is not diabetic
Row 19: The person is not diabetic
Row 20: The person is not diabetic
Row 21: The person is not diabetic
Row 22: The person is not diabetic
Row 23: The person is not diabetic
Row 24: The person is not diabetic
Row 25: The person is not diabetic
Row 26: The person is not diabetic
Row 27: The person is diabetic
Row 28: The person is not diabetic
Row 29: The person is not diabetic
Row 30: The person is 

Burada yapılan işlem, **eğitim (X_train) veya test (X_test) verisi** üzerinde modelin tahminler yapması ve her bir veri satırını sınıflandırmasıdır. Adım adım ne yapıldığına bakalım:

### 1. **Veri Seçimi:**
   ```python
   X_data = X_train  # ya da X_test
   ```
   Burada, veri kümesi (`X_train` veya `X_test`) seçilmektedir. Modeli test etmek için `X_train` ya da `X_test` kullanılabilir. Bu veriler, modelin tahmin yapmasını sağlamak için kullanılacaktır.

### 2. **Veri Standardizasyonu:**
   ```python
   X_data_standardized = scaler.transform(X_data)
   ```
   Bu adımda, seçilen veri (`X_data`, yani `X_train` veya `X_test`), daha önce eğitilen `scaler` (StandardScaler) kullanılarak standardize edilmektedir. Standardizasyon işlemi, her özelliğin (column) ortalamasının 0 ve standart sapmasının 1 olmasını sağlar. Bu, modelin daha hızlı ve doğru öğrenmesini sağlamak için gereklidir, çünkü bazı özellikler diğerlerinden daha büyük ölçeklerde olabilir.

### 3. **Tahmin Yapma:**
   ```python
   predictions = classifier.predict(X_data_standardized)
   ```
   Bu adımda, eğitimli olan **SVC (Support Vector Classifier)** modelimiz (`classifier`), standardize edilmiş veri üzerinde tahmin yapmaktadır. Modelin her bir örneğe ilişkin çıktısı, **0 (diabetik olmayan)** ya da **1 (diabetik)** olacaktır.

### 4. **Sonuçların Yazdırılması:**
   ```python
   for i, prediction in enumerate(predictions):
       if prediction == 0:
           print(f"Row {i+1}: The person is not diabetic")
       else:
           print(f"Row {i+1}: The person is diabetic")
   ```
   Burada, modelin her tahmini için bir döngü (`for` döngüsü) kullanılarak, tahminlerin sonucu ekrana yazdırılmaktadır:
   - **`i`**: Satırın numarasını belirtir.
   - **`prediction`**: Modelin yaptığı tahmin (`0` veya `1`).
   - Eğer tahmin **0** ise, kişi **diabetik değil** olarak değerlendirilir.
   - Eğer tahmin **1** ise, kişi **diabetik** olarak değerlendirilir.

   Her tahmin sonucunda, modelin çıktısı satır numarasıyla birlikte ekrana yazdırılır, örneğin:
   - "Row 1: The person is not diabetic"
   - "Row 2: The person is diabetic"
   - vb.

### Özet:
Bu kod parçası, bir veri seti üzerinde eğitimli modelin tahminlerini gerçekleştirir ve her bir örnek için, kişinin diyabetik olup olmadığı hakkında bir çıktı verir. olup olmadığı hakkında bir çıktı verir.onuç (diyabetli olup olmadığı) kullanıcıya gösterilir.